In [1]:
import boto.ec2
conn = boto.ec2.connect_to_region("us-east-1")

In [2]:
instances = conn.get_all_instances(filters={'key_name': 'dse-keypair-us-east-1'})

In [3]:
i = instances[-1].instances[0]
i.ip_address

u'54.209.121.205'

In [30]:
# from boto import kinesis

# kinesis = kinesis.connect_to_region("us-east-1")

# kinesis.list_streams()

{u'HasMoreStreams': False, u'StreamNames': []}

In [5]:
# import boto3

# client = boto3.client('cloudformation')

# response = client.describe_stacks(StackName='dse-stack')

In [51]:
# client.list_stack_resources(StackName='dse-stack')[u'StackResourceSummaries']

In [52]:
# response['Stacks'][0]

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster([i.ip_address])

In [9]:
session = cluster.connect()

In [10]:
session.execute("CREATE KEYSPACE demo WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

In [11]:
session = cluster.connect('demo')

In [12]:
session.execute("""CREATE TABLE users (
                    firstname text,
                    lastname text,
                    age int,
                    email text,
                    city text,
                PRIMARY KEY (lastname));""")

In [13]:
session.execute("""INSERT INTO users (lastname, age, city, email, firstname) 
                VALUES ('Jones', 35, 'Austin', 'bob@example.com', 'Bob')""");

In [14]:
result = session.execute("SELECT * FROM users WHERE lastname='Jones';")[0]
print result.firstname, result.age

Bob 35


In [16]:
session.execute("""CREATE TABLE example (
                    id int PRIMARY KEY,
                    tupleval tuple<int, text>,
                    numbers set<int>,
                    words list<text>
                );""");

In [26]:
data = '{"id": 0, "tupleval": [1, "abc"], "numbers": [1, 2, 3], "words": ["a", "b", "c"]}'

In [27]:
prepared_stmt = session.prepare("INSERT INTO example JSON ?;")
bound_stmt = prepared_stmt.bind([data])
stmt = session.execute(bound_stmt)